# Simulations for WAVEFIER

In [5]:
import os
import subprocess
import yaml
import sys

from astropy.io import fits
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm
import random
from scipy.stats import norm
import glob
from pathlib import Path

from matplotlib import pyplot as plt
import seaborn as sns

In [151]:
sns.set()
tqdm.pandas()

# Importing Data

In [222]:
events = pd.read_csv("../GammaCatalog_O5/events.csv")

# Simulate Observations with GW Toy

In [8]:
sys.path.append("..")
from gravitational_wave_toy import gwobserve as gw

## Select Random Delay for each event

In [82]:
# let's let the seed = the number of events for reproducibility

seed = len(events)

In [223]:
rng = np.random.default_rng(seed)

delays = []

for _ in range(len(events)):
    delays.append(rng.integers(0, 601))
    
delays = np.array(delays)

In [224]:
# add delays to events table

events["delay"] = delays

## Import Sensitivity
- CTA South
- z20
- no EBL

In [225]:
sens_file = "/Users/jarred/Documents/grbsens/notebooks/grbsens_output/no_ebl/grbsens_results/grbsens-5.0sigma_t1s-t16384s_irf-South_z20_0.5h.txt"

In [226]:
# Set up sensitivity
sens = gw.Sensitivity(
    sens_file,
    min_energy=0.3,     # as determined by the range of the IRFs
    max_energy=10000,
)

In [258]:
def observe_wavefier(row, sensitivity):
    
    filename = f"../GammaCatalog_O5/catO5_{row.event_id}.fits"
    
    grb = gw.GRB(filename)
    
    res = grb.observe(sensitivity=sensitivity, start_time=row.delay, target_precision=1, max_time=3600)
    
    return res["seen"], res["obs_time"], res["end_time"]

In [228]:
events[["seen", "obs_time", "end_time"]] = events.progress_apply(observe_wavefier, sensitivity=sens, axis=1, result_type="expand")

100%|██████████| 2307/2307 [03:15<00:00, 11.81it/s]


In [253]:
events.to_csv("wavefier_gw.csv", index=False)

## Try with all delays at 60s

In [255]:
events_60s = pd.read_csv("../GammaCatalog_O5/events.csv")

In [259]:
def observe_wavefier_60s(row, sensitivity):
    
    filename = f"../GammaCatalog_O5/catO5_{row.event_id}.fits"
    
    grb = gw.GRB(filename)
    
    res = grb.observe(sensitivity=sensitivity, start_time=60, target_precision=1, max_time=3600)
    
    return res["seen"], res["obs_time"], res["end_time"]

In [260]:
events_60s[["seen", "obs_time", "end_time"]] = events_60s.progress_apply(observe_wavefier_60s, sensitivity=sens, axis=1, result_type="expand")

100%|██████████| 2307/2307 [04:18<00:00,  8.92it/s]


In [268]:
events_60s.to_csv("wavefier_gw_60s.csv", index=False)

In [273]:
events_60s[~events_60s.seen]

,event_id,long,lat,eiso,dist,angle,time_utc,seen,obs_time,end_time
0,0,5.923,1.053,2.860000e+49,263000.0,63.682,2012-06-29 01:24:30.971,False,-1,-1
2,2,1.653,-0.551,5.730000e+47,909000.0,59.989,2012-01-16 13:16:18.682,False,-1,-1
3,3,0.749,1.098,6.660000e+48,1290000.0,55.033,2012-09-10 11:15:45.680,False,-1,-1
6,6,2.958,-0.543,4.500000e+51,315000.0,79.246,2012-08-22 07:26:53.204,False,-1,-1
9,9,4.450,-0.501,5.300000e+47,389000.0,85.783,2011-10-05 09:35:28.464,False,-1,-1
...,...,...,...,...,...,...,...,...,...,...
2300,2300,2.761,0.829,6.120000e+51,318000.0,62.799,2012-03-27 15:53:07.042,False,-1,-1
2301,2301,1.924,-0.882,9.720000e+50,319000.0,64.025,2011-11-25 05:41:28.700,False,-1,-1
2302,2302,3.211,-0.385,1.320000e+51,971000.0,61.195,2011-09-28 05:31:34.925,False,-1,-1
2303,2303,4.022,1.049,6.180000e+47,548000.0,62.712,2012-07-05 22:51:02.557,False,-1,-1


```yml
Summary of results, all with a delay of 60s:
- Total number of events: 2307
- detected within 1h: 1016 (44%)
- not detected: 1291 (56%)

- average detection time: 182s (3min)
- number of events detected in ~1s: 662 (~28% of all events, 65% of detected events)

- average delay of detected events: 303s
- average delay of non-detected events: 293s

- average angle of detected events: 27º
- average angle of non-detected events: 52º
```